In [1]:
from rmsd_calculator import calculate_rmsd_between_mols

# 假设两个文件中的分子是成对匹配的
rmsd_values = []
docked_mol = "/home/guest/AI_docking/EquiBind-main/data/results/output/Abl/lig_equibind_corrected.sdf"
reference_mol = "/home/guest/AI_docking/EquiBind-main/data/results/output/Abl/lig_equibind_corrected.sdf"
rmsd = calculate_rmsd_between_mols(docked_mol, reference_mol)
rmsd_values.append(rmsd)
print(f"RMSD: {rmsd}")

# for docked_mol, reference_mol in zip(docked_supplier, reference_supplier):
#     if docked_mol is not None and reference_mol is not None:
#         rmsd = calculate_rmsd_no_align(docked_mol, reference_mol)
#         rmsd_values.append(rmsd)
#         print(f"RMSD: {rmsd}")

# 输出 RMSD 的结果
print(f"Average RMSD: {sum(rmsd_values) / len(rmsd_values)}")

ImportError: cannot import name 'calculate_rmsd_between_mols' from 'rmsd_calculator' (/data/ZengXincheng/projects/docking_analysis_tool/rmsd_calculator.py)

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem

def calculate_rmsd(sdf_file1: str, sdf_file2: str) -> float:
    """
    计算两个SDF文件中相同分子构象之间的RMSD值。

    参数:
    sdf_file1 (str): 第一个SDF文件的路径。
    sdf_file2 (str): 第二个SDF文件的路径。

    返回:
    float: 两个分子构象之间的RMSD值。如果出现错误或分子不匹配，返回None。
    """
    try:
        # 读取第一个SDF文件中的分子
        mol_supplier1 = Chem.SDMolSupplier(sdf_file1)
        mol1 = mol_supplier1[0]
        
        # 读取第二个SDF文件中的分子
        mol_supplier2 = Chem.SDMolSupplier(sdf_file2)
        mol2 = mol_supplier2[0]
        
        if mol1 is None or mol2 is None:
            raise ValueError("其中一个SDF文件未能正确加载分子")

        # 为两个分子生成3D坐标
        AllChem.EmbedMolecule(mol1)
        AllChem.EmbedMolecule(mol2)

        # 计算RMSD值
        rmsd = AllChem.GetBestRMS(mol1, mol2)
        
        return rmsd
    except Exception as e:
        print(f"计算RMSD时出错: {e}")
        return None


In [6]:
from rmsd_calculator import GetBestRMSD
sdf_file1 = "/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/5MO4_asciminib/AY7_asciminib.sdf"
sdf_file2 = "/home/guest/AI_docking/EquiBind-main/data/results/output/5MO4_asciminib/lig_equibind_corrected.sdf"


# here, rms is Fitted, rmsd is NOT Fit!!!
GetBestRMSD(sdf_file1,sdf_file2)

(1.0731755995923544, 76.6305096423005)

In [42]:
sdf_file1 = "/home/guest/AI_docking/EquiBind-main/test_docking/test_docking_stage2/ABL1/5MO4_asciminib/AY7_asciminib.sdf"
sdfmol = Chem.SDMolSupplier(sdf_file1)
mol = next(iter(sdfmol), None)
print(type(mol))

<class 'rdkit.Chem.rdchem.Mol'>


In [45]:
from rdkit import Chem
# m = Chem.MolFromSmiles('OC1C2C1CC2')
m2 = Chem.AddHs(mol)
print("m Smiles:",Chem.MolToSmiles(mol))
print("m2 Smiles:",Chem.MolToSmiles(m2))
print("num ATOMs in m:",mol.GetNumAtoms())
print("num ATOMs in m2:",m2.GetNumAtoms())

m Smiles: O=C(Nc1ccc(OC(F)(F)Cl)cc1)c1cnc(N2CC[C@@H](O)C2)c(-c2ccn[nH]2)c1
m2 Smiles: [H]O[C@@]1([H])C([H])([H])N(c2nc([H])c(C(=O)N([H])c3c([H])c([H])c(OC(F)(F)Cl)c([H])c3[H])c([H])c2-c2c([H])c([H])nn2[H])C([H])([H])C1([H])[H]
num ATOMs in m: 31
num ATOMs in m2: 49


In [38]:
# 例子分子
# mol = Chem.MolFromSmiles('CCCCO')  # 乙醇

# 选取重原子（原子序数大于1）
heavy_atoms = [atom for atom in mol.GetAtoms()]

# 打印重原子的信息
for atom in heavy_atoms:
    print(f'原子编号: {atom.GetIdx()}, 元素: {atom.GetSymbol()}')

原子编号: 0, 元素: C
原子编号: 1, 元素: C
原子编号: 2, 元素: C
原子编号: 3, 元素: C
原子编号: 4, 元素: C
原子编号: 5, 元素: C
原子编号: 6, 元素: C
原子编号: 7, 元素: C
原子编号: 8, 元素: C
原子编号: 9, 元素: C
原子编号: 10, 元素: C
原子编号: 11, 元素: C
原子编号: 12, 元素: C
原子编号: 13, 元素: C
原子编号: 14, 元素: N
原子编号: 15, 元素: C
原子编号: 16, 元素: C
原子编号: 17, 元素: C
原子编号: 18, 元素: N
原子编号: 19, 元素: C
原子编号: 20, 元素: C
原子编号: 21, 元素: C
原子编号: 22, 元素: N
原子编号: 23, 元素: O
原子编号: 24, 元素: O
原子编号: 25, 元素: O
原子编号: 26, 元素: N
原子编号: 27, 元素: N
原子编号: 28, 元素: F
原子编号: 29, 元素: F
原子编号: 30, 元素: Cl


In [18]:
for i in sdfmol:
    print(i)

In [9]:
smi='CC(C)OC(=O)C(C)NP(=O)(OCC1C(C(C(O1)N2C=CC(=O)NC2=O)(C)F)O)OC3=CC=CC=C3'
from rdkit import Chem
from rdkit.Chem import AllChem
mol = Chem.MolFromSmiles(smi)
print(type(mol))

<class 'rdkit.Chem.rdchem.Mol'>


In [23]:
from rdkit import Chem
from rdkit.Chem import rdMolAlign

sdf_file2 = sdf_file1 = "/home/guest/AI_docking/EquiBind-main/data/results/output/5MO4_asciminib/lig_equibind_corrected.sdf"

from rdkit import Chem
from rdkit.Chem import AllChem

def calculate_rmsd_no_align(mol1, mol2):
    # 直接计算RMSD, 不进行对齐
    AllChem.GetBestRMS(mol1, mol2)
    rmsd = AllChem.GetConformerRMS(mol1, mol2, prealigned=True)
    return rmsd

# 读取对接后的分子和真实分子的SDF文件
docked_supplier = Chem.SDMolSupplier(sdf_file2)
reference_supplier = Chem.SDMolSupplier(sdf_file2)

calculate_rmsd_no_align(docked_supplier, reference_supplier)


ArgumentError: Python argument types in
    rdkit.Chem.rdMolAlign.GetBestRMS(SDMolSupplier, SDMolSupplier)
did not match C++ signature:
    GetBestRMS(RDKit::ROMol {lvalue} prbMol, RDKit::ROMol {lvalue} refMol, int prbId=-1, int refId=-1, boost::python::api::object map=None, int maxMatches=1000000, bool symmetrizeConjugatedTerminalGroups=True, boost::python::api::object weights=[])

In [13]:
mol_supplier1[0]

[11:14:39] ERROR: Counts line too short: 'SMALL' on line158
[11:14:39] ERROR: moving to the beginning of the next molecule
